In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import numpy as np
import os
from skimage import io
from pathlib import Path
import time

# PATH VARIABLES

In [61]:
from google.colab import drive
drive.mount('/content/drive')
test_folder = '/content/drive/MyDrive/Colab Notebooks/FinalProjectStuff/test'
train_folder = '/content/drive/MyDrive/Colab Notebooks/FinalProjectStuff/train'
saved_weights_path = '/content/drive/MyDrive/Colab Notebooks/FinalProjectStuff/savedModel.tar'
csv_path = '/content/drive/MyDrive/Colab Notebooks/FinalProjectStuff/PredictionCSV.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
class CustomDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, root_dir, transform=None):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.data = []

        for filename in os.listdir(self.root_dir):
            f = os.path.join(self.root_dir, filename)
            # checking if it is a file
            if Path(f).is_file() and Path(f).suffix == '.jpg':
                test, ext = os.path.splitext(filename)
                image = io.imread(f)
                image = self.grabImage(image)
                self.data.append((image, None, Path(f).name))

            elif not Path(f).is_file():
                for filename_real in os.listdir(f):
                    realPath = os.path.join(f, filename_real)
                    if Path(realPath).is_file() and Path(realPath).suffix == '.jpg':
                        image = io.imread(realPath)
                        image = self.grabImage(image)
                        self.data.append((image, filename, Path(realPath).name))

    def __len__(self):
        return len(self.data)

    def grabImage(self, image):
        newImage = image
        if self.transform:
            newImage = self.transform(image)
        return newImage


    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        #print(idx)
        path, label, name = self.data[idx]
        image = self.grabImage(path)
        #print(path)
        if label:
            return image, int(label), name
        return image, name # kinda ugly, is it better to just return None?

In [69]:
trans_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
])

trans_base = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(size = (224, 224), antialias=None)
])

train_dataset = CustomDataset(train_folder, trans_base)


In [70]:
# size 1079, aka 83 * 13 - split maybe 11 - 2 ?
# train_dataloader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, lengths = [83 * 11, 83 * 2])  #size of train and val dataset

train_dataset.dataset.transform = trans_train
val_dataset.dataset.transform = None

train_dataloader = DataLoader(train_dataset, batch_size = 83, shuffle = True)
val_dataloader = DataLoader(val_dataset, batch_size = 83, shuffle = False)

# images, labels = next(iter(train_dataloader))



In [56]:
print(len(test_dataset))
print(len(train_dataset))
print(len(val_dataset))
print(len(train_dataset.indices))


1036
913
166
913


In [25]:
def imshow(img):
    img = img / 2 + 0.5  # Unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
# heck yea, done so far p good :D
# imshow(torchvision.utils.make_grid(images))

In [71]:
weights = models.ResNet18_Weights.DEFAULT
preprocess = weights.transforms()

model = models.resnet18(weights = weights)

In [72]:
for param in model.parameters():
    param.requires_grad = False

fc_in = model.fc.in_features

model.fc = nn.Sequential(
    nn.Linear(fc_in, 512),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(512, 100),
    nn.LogSoftmax(dim=1) # For using NLLLoss()
)

In [75]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters())

In [76]:
num_epochs = 25
for epoch in range(num_epochs):
    epoch_start = time.time()
    model.train()
    for inputs, labels, name in train_dataloader:
        ###### Your code starts here. ######
        # do forward-backward propogation, and update the model
        optimizer.zero_grad()  # reset weights to 0
        yPred = model(inputs)  # forwards pass
        loss = criterion(yPred, labels)  # loss between yPred and ground truths
        loss.backward() # backprop
        # model.net[0].weight.grad[0,0,0,0] = -10
        optimizer.step()

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    acc = 0
    with torch.no_grad():
        for inputs, labels, name in val_dataloader:
            outputs = model(inputs)
            # print(outputs)
            ###### Your code starts here. ######
            # predictions = model.smx(outputs)
            predicted = torch.argmax(outputs, dim = 1)
            ###### Your code starts here. ######
            val_loss += criterion(outputs, labels)
            correct += (predicted == labels.squeeze()).sum().item()
            total += len(labels)

    torch.save({ "model_state_dict": model.state_dict(),
                 "optimizer_state_dict": optimizer.state_dict()
                 }, saved_weights_path)
    epoch_end = time.time()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {val_loss/len(val_dataloader):.4f}, Accuracy: {correct/total:.4f}')
    print("Time taken", epoch_end - epoch_start)


Epoch 1/25, Loss: 4.0337, Accuracy: 0.0843
Time taken 122.55571222305298
Epoch 2/25, Loss: 3.3354, Accuracy: 0.1747
Time taken 123.04635190963745
Epoch 3/25, Loss: 2.7706, Accuracy: 0.2771
Time taken 124.31998181343079
Epoch 4/25, Loss: 2.3951, Accuracy: 0.4157
Time taken 130.0421793460846
Epoch 5/25, Loss: 2.2085, Accuracy: 0.4398
Time taken 122.49460339546204
Epoch 6/25, Loss: 2.0242, Accuracy: 0.4699
Time taken 126.16724801063538
Epoch 7/25, Loss: 1.8920, Accuracy: 0.5000
Time taken 136.6195890903473
Epoch 8/25, Loss: 1.8043, Accuracy: 0.5482
Time taken 124.78062629699707
Epoch 9/25, Loss: 1.7283, Accuracy: 0.5301
Time taken 126.22352576255798
Epoch 10/25, Loss: 1.6718, Accuracy: 0.5542
Time taken 124.44701743125916
Epoch 11/25, Loss: 1.6278, Accuracy: 0.5602
Time taken 127.5014877319336
Epoch 12/25, Loss: 1.6214, Accuracy: 0.5422
Time taken 126.18478846549988


KeyboardInterrupt: ignored

In [77]:
def evaluate_image(image, model):
    outputs = model(image)
    pred = torch.argmax(outputs, dim = 1)
    return pred

In [ ]:
import csv

test_dataset = CustomDataset(test_folder, trans_base)
test_dataloader = DataLoader(test_dataset, shuffle = False) # picked because divides nicely into 4
test_dataset.transform = None


model = models.resnet18()
for parameter in model.parameters():
    parameter.requires_grad = False

fc_in = model.fc.in_features

model.fc = nn.Sequential(
    nn.Linear(fc_in, 512),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(512, 100),
    nn.LogSoftmax(dim=1) # For using NLLLoss()
)
optimizer = optim.Adam(model.parameters())
checkpoint = torch.load(saved_weights_path)

model.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

with open(csv_path, 'w', newline='') as f:
    writer = csv.writer(f)
    for image, name in test_dataloader:
        pred = evaluate_image(image, model).item()
        print(pred)
        writer.writerow([name, pred])


9
27
27
15
79
1
57
9
24
17
1
83
64
27
22
30
98
68
91
56
24
24
3
24
28
4
56
1
88
24
40
94
1
9
62
1
8
9
1
1
9
32
56
32
97
9
9
30
22
62
1
3
9
23
62
24
1
3
17
24
9
47
9
8
40
79
3
98
41
9
85
74
24
24
24
9
1
9
9
27
1
24
24
9
56
32
24
27
14
9
9
40
79
24
9
98
98
24
24
1
97
27
27
56
1
85
57
40
3
27
57
24
40
95
9
9
98
79
24
79
24
30
27
9
1
9
41
1
91
24
9
24
1
24
24
62
24
98
24
71
24
22
29
32
91
27
62
9
9
1
98
50
62
1
27
17
90
24
32
47
62
1
98
32
24
5
1
7
1
79
27
1
40
18
91
27
14
27
88
57
98
32
9
24
24
9
47
3
83
9
62
17
50
62
24
40
9
8
68
1
91
9
1
27
30
57
27
24
9
1
91
57
79
1
24
57
3
3
24
91
40
24
1
1
24
1
40
1
71
24
62
32
30
79
98
1
88
79
3
27
98
22
38
9
1
1
98
88
74
71
91
24
1
56
9
9
64
9
27
9
91
3
24
9
1
47
88
9
65
80
62
3
24
98
74
52
1
9
1
91
27
98
3
57
79
1
3
9
1
10
8
9
17
24
22
1
24
27
8
64
68
22
1
15
64
71
56
23
64
57
88
32
9
47
1
27
12
17
83
17
27
67
40
8
27
3
40
91
74
22
40
9
9
79
1
1
1
1
1
9
1
1
9
1
24
57
24
27
27
91
1
88
1
27
40
91
24
8
9
1
1
9
9
1
9
40
62
3
91
1
9
1
9
95
24
27
63
9
9